In [8]:
import pandas as pd
import pandas_ta as ta
import numpy as np
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import requests
import defs

In [ ]:
df.columns

In [13]:
fx_pairs = ['AUD_JPY', 'NZD_JPY', 'EUR_JPY', 'GBP_JPY', 'USD_JPY', 'AUD_USD', 'AUD_CAD', 'NZD_USD', 'EUR_USD', 'GBP_USD', 'USD_CAD', 'XAU_USD', 'BTC_USD', 'WTICO_USD']
timeframes = ['H1', 'H4', 'D']
session = requests.Session()
fx_dataframes = {}
trend_df_columns = ['Asset'] + [f'MOMO_{timeframe}' for timeframe in timeframes] + [f'PATI_{timeframe}' for timeframe in timeframes]

trend_data_list = []
total_bytes = 0

for instrument in fx_pairs:
    trend_data = {'Asset': instrument}
    for timeframe in timeframes:
        url = f'{defs.OANDA_URL}/instruments/{instrument}/candles'
        params = {'count': 60, 'granularity': timeframe, 'price': "M"}
        response = session.get(url, params=params, headers=defs.SECURE_HEADER)

        if response.status_code == 200:
            size_in_bytes = len(response.content)
            total_bytes += size_in_bytes
            data = response.json()
            my_data = [{'time': candle['time'], 'open': float(candle['mid']['o']), 'high': float(candle['mid']['h']), 'low': float(candle['mid']['l']), 'close': float(candle['mid']['c']), 'volume': candle['volume']} for candle in data['candles'] if candle['complete']]
            precision = 3 if 'JPY' in instrument else 5
            df = pd.DataFrame(my_data)
            df['time'] = pd.to_datetime(df['time'])
            df.set_index('time', inplace=True)
            df['SMA_FAST'] = ta.sma(df['close'], length=20).round(precision)
            df['SMA_SLOW'] = ta.sma(df['close'], length=50).round(precision)
            df['MOMO'] = np.where(df['SMA_FAST'] > df['SMA_SLOW'], 'Uptrend', 'Downtrend')
            df['RSI'] = ta.rsi(df['close'], length=14)
            fx_dataframes[(instrument, timeframe)] = df

            # Initialize variables for trend tracing logic
            df['Candle'] = np.select([df['close'] > df['open'], df['close'] < df['open']], ['Bull', 'Bear'], default='Doji')
            df['Trend'] = 'Neutral'
            df.at[df.index[0], 'Trend'] = 'Neutral'
            
            higher_high = None
            lower_low = None

            bullish_pullback_count = 0
            bearish_pullback_count = 0
            bullish_pullback = False
            bearish_pullback = False

            swing_high = None
            swing_low = None

            uptrend_lowest_pullback = None
            downtrend_highest_pullback = None

            previous_high = df['high'].iloc[0]
            previous_low = df['low'].iloc[0]

            current_trend = 'Neutral'
            df.iloc[0, df.columns.get_loc('Trend')] = current_trend


            df['Price_Status'] = None
            price_status = None

            # Iterating logic starts
            for i, row in df.iloc[1:].iterrows():
                # Insert trend tracing logic here and update df['Trend'] accordingly
                current_high = row['high']
                current_low = row['low']
                candle_type = row['Candle']
                current_close = row['close']

                # clear_output()
                # Trend rotation
                if current_trend == "Neutral":
                    if swing_low is None and swing_high is None:
                        swing_high = current_high
                        swing_low = current_low
                        higher_high = current_high
                        lower_low = current_low

                if current_close > swing_high and current_close > higher_high:
                    current_trend = 'Uptrend'
                    bullish_pullback_count = 0
                    bullish_pullback = False
                    bearish_pullback_count = 0
                    bearish_pullback = False
                    if uptrend_lowest_pullback is not None:
                        swing_low = uptrend_lowest_pullback
                elif current_close < swing_low and current_low < lower_low:
                    current_trend = 'Downtrend'
                    bearish_pullback_count = 0
                    bearish_pullback = False
                    bullish_pullback_count = 0
                    bullish_pullback = False
                    if downtrend_highest_pullback is not None:
                        swing_high = downtrend_highest_pullback
                # Finding the range in the beginning of the dataset

                df.at[i, 'Trend'] = current_trend

                if bullish_pullback or bearish_pullback:
                    price_status = 'Consolidation'
                else:
                    price_status = 'Extension'

                df.at[i, 'Price_Status'] = price_status
                # ================================================================================================== #

                if current_trend == 'Uptrend':
                    if current_high > previous_high and current_low > swing_high and not bullish_pullback:
                        if higher_high is not None:
                            higher_high = max(higher_high, current_high)
                        else:
                            higher_high = current_high
                    if current_close < higher_high and candle_type == 'Bear':
                        bullish_pullback_count += 1
                    if bullish_pullback_count >= 2:
                        bullish_pullback = True
                        swing_high = higher_high

                    if bullish_pullback and current_close > swing_low:
                        if current_low < previous_low:
                            uptrend_lowest_pullback = current_low

                elif current_trend == 'Downtrend':
                    if current_low < previous_low and current_low < swing_low and not bearish_pullback:
                        if lower_low is not None:
                            lower_low = min(lower_low, current_low)
                        else:
                            lower_low = current_low
                    if current_close > lower_low and candle_type == 'Bull':
                        bearish_pullback_count += 1
                    if bearish_pullback_count >= 2:
                        bearish_pullback = True
                        swing_low = lower_low

                    if bearish_pullback and current_close < swing_high:
                        if current_high > previous_high:
                            downtrend_highest_pullback = current_high

                df.at[i, 'Swing_high'] = swing_high
                df.at[i, 'Swing_low'] = swing_low
                previous_low = current_low
                previous_high = current_high
                
                # =================================================================================================================================
                

            # Extract the final 'Trend' value and update trend_data
            final_trend = df['Trend'].iloc[-1]
            column_name = f'PATI_{timeframe}'
            trend_data[column_name] = final_trend
            trend_data[f'MOMO_{timeframe}'] = df['MOMO'].iloc[-1]
        else:
            print(f"Failed to fetch data for {instrument} with granularity {timeframe}: {response.status_code}")
            column_name = f'PATI_{timeframe}'
            trend_data[column_name] = 'Error'

    trend_data_list.append(trend_data)

trend_df = pd.DataFrame(trend_data_list, columns=trend_df_columns)
trend_df.set_index('Asset', inplace=True)


total_kb = total_bytes / 1024
total_mb = round((total_kb / 1024), 2)

trend_df['Uptrend_Count'] = trend_df.apply(lambda row: (row == 'Uptrend').sum(), axis=1)
trend_df['Downtrend_Count'] = trend_df.apply(lambda row: (row == 'Downtrend').sum(), axis=1)
trend_df = trend_df.sort_index()

print(f'The total KB downloaded was {total_kb} ({total_mb})MB')



             MOMO_H1    MOMO_H4     MOMO_D    PATI_H1    PATI_H4     PATI_D
Asset                                                                      
AUD_JPY      Uptrend    Uptrend    Uptrend    Uptrend  Downtrend    Uptrend
NZD_JPY      Uptrend  Downtrend    Uptrend    Uptrend  Downtrend    Uptrend
EUR_JPY      Uptrend    Uptrend    Uptrend    Uptrend  Downtrend    Uptrend
GBP_JPY      Uptrend    Uptrend    Uptrend    Uptrend  Downtrend    Uptrend
USD_JPY      Uptrend    Uptrend    Uptrend    Uptrend  Downtrend    Uptrend
AUD_USD    Downtrend  Downtrend  Downtrend  Downtrend    Uptrend  Downtrend
AUD_CAD    Downtrend  Downtrend    Uptrend  Downtrend    Uptrend  Downtrend
NZD_USD    Downtrend  Downtrend    Uptrend  Downtrend  Downtrend  Downtrend
EUR_USD    Downtrend  Downtrend    Uptrend  Downtrend    Uptrend  Downtrend
GBP_USD    Downtrend  Downtrend    Uptrend  Downtrend    Uptrend    Uptrend
USD_CAD      Uptrend    Uptrend    Uptrend    Uptrend  Downtrend    Uptrend
XAU_USD     

In [18]:

total_kb = total_bytes / 1024
total_mb = round((total_kb / 1024), 2)

print(f'The total KB downloaded was {total_kb} ({total_mb})MB')


The total KB downloaded was 337.2763671875 (0.33)MB


In [23]:
trend_df


,MOMO_H1,MOMO_H4,MOMO_D,PATI_H1,PATI_H4,PATI_D,Uptrend_Count,Downtrend_Count
Asset,,,,,,,,
AUD_CAD,Downtrend,Downtrend,Uptrend,Downtrend,Uptrend,Downtrend,2,4
AUD_JPY,Uptrend,Uptrend,Uptrend,Uptrend,Downtrend,Uptrend,5,1
AUD_USD,Downtrend,Downtrend,Downtrend,Downtrend,Uptrend,Downtrend,1,5
BTC_USD,Downtrend,Downtrend,Uptrend,Downtrend,Uptrend,Uptrend,3,3
EUR_JPY,Uptrend,Uptrend,Uptrend,Uptrend,Downtrend,Uptrend,5,1
EUR_USD,Downtrend,Downtrend,Uptrend,Downtrend,Uptrend,Downtrend,2,4
GBP_JPY,Uptrend,Uptrend,Uptrend,Uptrend,Downtrend,Uptrend,5,1
GBP_USD,Downtrend,Downtrend,Uptrend,Downtrend,Uptrend,Uptrend,3,3
NZD_JPY,Uptrend,Downtrend,Uptrend,Uptrend,Downtrend,Uptrend,4,2


In [22]:
trend_df = trend_df.sort_index()

In [ ]:
fx_dataframes['AUD_CAD', 'H4'].tail()

In [3]:
uptrend_count = (trend_df.loc['AUD_USD'] == 'Uptrend').sum()
uptrend_count


1